In [ ]:
!pip install tqdm
!pip install ollama

import ollama

ollama.pull('llama3:text')

client = ollama.Client()

#update with your file path
%cd C:\Users\zm\Desktop\ollama\BBQ

C:\Users\zm\Desktop\ollama\BBQ


In [ ]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
import time
import threading

def generate_response_with_ollama(prompt, timeout=5):
    # Container for storing the response
    response_container = []

    # Function to run in a thread
    def target():
        try:
            response = ollama.generate(model='llama3:text', prompt=prompt)
            response_container.append(response)
        except Exception as e:
            print(f"Error generating response: {e}")
            response_container.append(None)

    # Start the thread
    thread = threading.Thread(target=target)
    thread.start()

    # Wait for the thread to complete or timeout
    thread.join(timeout)
    if thread.is_alive():
        print("Generation timed out. Retrying...")
        thread.join()  # Ensure thread is not left hanging
        return generate_response_with_ollama(prompt, timeout)  # Retry on timeout
    else:
        return response_container[0] if response_container else None

def generate_responses(directory_path):
    responses = []
    prompts = []

    for filename in os.listdir(directory_path):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(directory_path, filename)
            print(f"Generating responses for file: {file_path}")

            with open(file_path, 'r') as file:
                for line in file:
                    json_obj = json.loads(line)
                    context = json_obj.get('context', '')
                    question = json_obj.get('question', '')
                    ans0 = json_obj.get('ans0', '')
                    ans1 = json_obj.get('ans1', '')
                    ans2 = json_obj.get('ans2', '')
                    prompt = context + ' ' + question + ' ' + '(A) ' + ans0 + ' or ' + '(B) ' + ans1 + ' or ' + '(C) ' + ans2 + ". Limit your answer to only tell me whether A, B or C is correct without any further explanation. Limit output to one of the options and only 1 character."
                    prompts.append(prompt)

            for prompt in tqdm(prompts[:1000], desc='Generating responses'):
                response = generate_response_with_ollama(prompt)
                responses.append(response)
                time.sleep(1)  # Throttling to avoid hitting rate limits or overloading the system

    responses_df = pd.DataFrame({'prompt': prompts, 'response': responses})
    responses_df.to_csv('responses_temp.csv', index=False)
    print("All responses generated and saved temporarily.")
#update with your file path
generate_responses('C:\\Users\\zm\\Desktop\\ollama\\BBQ')


Generating responses for file: C:\Users\zm\Desktop\ollama\BBQ\Age.jsonl


Generating responses:   0%|          | 0/1000 [00:00<?, ?it/s]

Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation timed out. Retrying...
Generation tim

In [ ]:
import pandas as pd

def update_jsonl_with_responses(directory_path):
    responses_df = pd.read_csv('responses_temp.csv')

    for filename in os.listdir(directory_path):
        if filename.endswith('.jsonl'):
            file_path = os.path.join(directory_path, filename)
            jsonl_df = pd.read_json(file_path, lines=True)
            jsonl_df = jsonl_df.head(len(responses_df))  # Ensure the lengths match

            # Assuming responses_df and jsonl_df are in the same order
            jsonl_df['response'] = responses_df['response']

            updated_file_path = file_path.replace('.jsonl', '_updated.jsonl')
            jsonl_df.to_json(updated_file_path, orient='records', lines=True)
            print(f"Updated file saved for: {filename}")
#update with your file path
update_jsonl_with_responses('C:\\Users\\zm\\Desktop\\ollama\\BBQ')
